# Creating the Video with openCV

Import and preprocess the data cubes that were downloaded with the corresponding script.

In [3]:
import numpy as np
import cv2
sat = 'S2'
layer = 'AGRICULTURE'
location = 'ALJAWF'
lon = 23.32
lat = 24.16
if lat >= 0:
    filename = f'{sat}_{location}_{layer}_{lon}E_{lat}N'
else:
    lat = abs(lat)
    filename = f'{sat}_{location}_{layer}_{lon}E_{lat}S'
outputname=f'smooth_{location}_{layer}'
print(outputname)

smooth_ALJAWF_AGRICULTURE


In [4]:
date = np.load('Data/'+filename+'_days.npy')
data = np.load('Data/'+filename+'.npy')
dataF = []
dateF = []
dummy = None

# get rid of double images
for i in range(len(date)):
    
    print(f'{i}/{len(date)}', end='\r')
    # sort out same days
    if dummy == date[i]:
        pass
    else:
        dummy = date[i]
        dateF.append(date[i])
        dataF.append(data[i])

print(f' # days: {len(dateF)}')
dataF = np.asarray(dataF)

# inverse the data from RGB to BGR for openCV
try:
    dummy = dataF[:,:,:,::-1]
    dataInv = dummy[::-1,:,:,:]
except:
    dataInv = dataF[::-1,:,:]
dateInv = dateF[::-1]

del(dataF)

 # days: 20


The following cell is needed if there are a lot of images with a high amount of white pixels, i.e. uncomplete scenes. Those images will not be included in the video.


The user specifies a percentage of number of pixels out of the total amount of pixels to be a threshold. Every image with a higher content of white pixels than the threshold will be left out of the video.

In [5]:
# only take scenes with amount of white pixels <= certain threshold
filtered = []
dateFiltered = []
channels = None
try:
    number, height , width , channels =  dataInv.shape
except:
    number, height , width =  dataInv.shape

threshold = 90 # in percent, 100% means every image will be used.
maxWhitePix = height * width * threshold/100

if channels != None:
    for i in range(len(dataInv)):
        if np.where((dataInv[i] >= 254).all(axis=2))[0].size <= maxWhitePix:
            filtered.append(dataInv[i])
            dateFiltered.append(dateInv[i])

else:
    for i in range(len(dataInv)):
        if np.where((dataInv[i] >= 254))[0].size <= maxWhitePix:
            filtered.append(dataInv[i])
            dateFiltered.append(dateInv[i])
            
            
print('total number of images:', number)
print('images without white pixels:', len(filtered))
filtered = np.array(filtered)

total number of images: 20
images without white pixels: 19


fill areas without data (i.e. areas with three values above 254) with the data from the previous frame (i.e. date)

In [6]:
smooth = []
datesmooth = []


if channels != None:
    for i in range(1, len(filtered)):
        x, y = np.where((filtered[i] >= 260).all(axis=2))
        filtered[i, x, y] = filtered[i-1, x, y]
        smooth.append(filtered[i])
        datesmooth.append(dateFiltered[i])
else:
    for i in range(1, len(filtered)):
        x, y = np.where(filtered[i] >= 260)
        filtered[i, x, y] = filtered[i-1, x, y]
        smooth.append(filtered[i])
        datesmooth.append(dateFiltered[i])
    smooth = np.repeat(np.array(smooth)[:,:,:,np.newaxis], 3, axis=3)
smooth = np.array(smooth)

In [8]:

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video = None
fps = 25
video = cv2.VideoWriter(f'Videos/{outputname}.avi',fourcc, fps, (width,height))

font = cv2.FONT_HERSHEY_SIMPLEX
fontSize = 0.5
color = (255,255,255)
thickness = 1
linetype = 0

i=0
starting = True
prev_frame = np.uint8([250])

while(True):
    
    frame = smooth[i]
    print(f'{i}/{len(smooth)}', end='\r')

    cv2.rectangle(smooth[i],(0,0),(200,80),(1,1,1), thickness=-1)
    # putText(image as array, text as string, position in the image, ..., ..., color as bgr?, ., .)
    cv2.putText(smooth[i], datesmooth[i],             (10,25), font, fontSize, color, thickness, linetype)
    cv2.putText(smooth[i], f'lon: {lon} / lat: {lat}',(10,40), font, fontSize, color, thickness, linetype)
    cv2.putText(smooth[i], f'layer: {layer}',         (10,55), font, fontSize, color, thickness, linetype)
    
    if starting==True:
        prev_frame = frame
        starting = False
    else:
        #video.write(frame)
        for l in range(1,20):
            weight = (20-l)/20
            #get the blended frames in between
            mid_frame = cv2.addWeighted(prev_frame,weight,frame,1-weight,0)
            video.write(mid_frame)
        
        for m in range(int(round(fps/2.))):
            # add half a second, filled with unprocessed scenes between the blended frames
            video.write(frame)
        prev_frame = frame
    i += 1
    if i == len(smooth):
        break
        

video.release()
cv2.destroyAllWindows()